In [ ]:
dictofweights={'S&P':18.2,'Ftse100':15.2,'AEX':17.8,"Dow jones":1.1,'Ftse mib':1.4}
yearly=pd.DataFrame(index=[datetime.datetime.now().year])
monthly=pd.DataFrame(index=[pd.to_datetime(str(datetime.datetime.today().year)+'-'+str(datetime.datetime.today().month)+'-01',errors='coerce')+BMonthEnd(1)])

def index (df,dictofweights,typeofdate):

    
    x = { df.column : df.index[df[df.column]>0][0] for df.column in df}
    weights=[]
    for k in dictofweights.values():
        weights.append(k/sum(dictofweights.values()))
    
    Z=pd.DataFrame(pd.Series(weights),columns=['weighting'])
    Z.index= pd.MultiIndex.from_tuples(x.items())
    Z=Z.sort_index(level=1)

    P=Z.unstack()
    P = P.T.xs('weighting', axis=0, drop_level=True)
    P=P.append(typeofdate)
    
    P=P.T
    weightendofperiod=P.sum(axis=0).cumsum()
    sumweights=P.T.sum(axis=1)
    P=P.ffill(axis=1)
    
    P=P.T.div(weightendofperiod,axis=0)
    P=P.fillna(0)
    df1=df
    df3=pd.DataFrame()
    
    for  column in df1:
        firstdate=(np.where(df1[column].notnull())[0][0])
        df3=df3.append(df1[column]/df1[column].iloc[firstdate])
    df3=df3.T
    K=0
    lengthp=len(P)
    df2=pd.DataFrame(columns=P.columns)
    while K < lengthp-1:
        df2=df2.append((df3.loc[P.index[K]:P.index[K+1]]*P.iloc[K]))
        K+=1
    df2 = df2[~df2.index.duplicated(keep='last')]
    df4=df2.sum(axis=1)
    df4=pd.DataFrame(df4,columns=['index'])
    
    return df4
dfindex=index(indexprices,dictofweights,monthly)
